In [1]:
# !~/tf-nvidia/bin/pip3 install git+https://github.com/huseinzol05/tensor2tensor.git --no-deps

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

2022-08-02 14:57:43.890732: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0



/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import youtokentome as yttm

In [5]:
bpe = yttm.BPE(model='rumi-jawi.yttm')

In [6]:
class Encoder:
    def __init__(self, bpe):
        self.bpe = bpe
        self.vocab_size = len(self.bpe.vocab())

    def encode(self, s):
        s = self.bpe.encode(s, output_type=yttm.OutputType.ID)
        s = [i + [1] for i in s]
        return s

    def decode(self, ids, strip_extraneous=False):
        return self.bpe.decode(list(ids))[0]

In [7]:
@registry.register_problem
class Jawi(text_problems.Text2TextProblem):
    @property
    def approx_vocab_size(self):
        return 32000

    @property
    def is_generate_per_split(self):
        # generate_data will shard the data into TRAIN and EVAL for us.
        return False

    @property
    def dataset_splits(self):
        return [
            {'split': problem.DatasetSplit.TRAIN, 'shards': 200},
            {'split': problem.DatasetSplit.EVAL, 'shards': 1},
        ]

    def feature_encoders(self, data_dir):
        s_encoder = Encoder(bpe)
        return {'inputs': s_encoder, 'targets': s_encoder}

In [8]:
PROBLEM = 'jawi'
problem = problems.problem(PROBLEM)

In [9]:
class Model:
    def __init__(self, HPARAMS = "transformer_small", DATA_DIR = 't2t/data2'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.beam_size = tf.placeholder(tf.int32, shape=(), name = 'beam_size')
        self.temperature = tf.placeholder(tf.float32, shape=(), name = 'temperature')
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        maxlen_decode = 50 + tf.reduce_max(self.X_seq_len)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        print(features)
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        logits, _ = translate_model(features)
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, maxlen_decode, beam_size=self.beam_size, 
                top_beams=1, alpha=self.temperature)["outputs"]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')

In [10]:
import tensorflow as tf

In [11]:
ckpt_path = tf.train.latest_checkpoint('t2t-rumi-jawi/small')
ckpt_path

't2t-rumi-jawi/small/model.ckpt-100000'

In [12]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()


2022-08-02 14:57:50.062777: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2496000000 Hz
2022-08-02 14:57:50.063125: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x588b000 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-02 14:57:50.063140: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-02 14:57:50.064391: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-02 14:57:50.154219: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 14:57:50.155021: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3441d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-08-02 14:57

2022-08-02 14:57:50.375483: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1206] Device interconnect StreamExecutor with strength 1 edge matrix:
2022-08-02 14:57:50.375511: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212]      0 
2022-08-02 14:57:50.375515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1225] 0:   N 
2022-08-02 14:57:50.375681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 14:57:50.376424: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 14:57:50.377122: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1351] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 20283 MB memory) -> physical GPU (device: 0, name: NVIDIA GeF

{'inputs': <tf.Tensor 'ExpandDims_1:0' shape=(?, ?, 1, 1) dtype=int32>, 'targets': <tf.Tensor 'ExpandDims_3:0' shape=(?, ?, 1, 1) dtype=int32>, 'target_space_id': <tf.Tensor 'Const_1:0' shape=() dtype=int32>}
INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Transforming feature 'targets' with symbol_modality_32000_256.targets_bottom


INFO:tensorflow:Building model body


INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_32000_256.top


INFO:tensorflow:Transforming body output with symbol_modality_32000_256.top


In [13]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from t2t-rumi-jawi/small/model.ckpt-100000


INFO:tensorflow:Restoring parameters from t2t-rumi-jawi/small/model.ckpt-100000


In [14]:
import json

train_X, train_Y = [], []
with open('../jawi-rumi/jawi-rumi-news-full.test') as fopen:
    for line in fopen:
        d = json.loads(line)
        train_Y.append(d[1])
        train_X.append(d[0])

In [15]:
train_X[:5]

['laporan sebuah portal berita baru-baru ini memetik seorang ahli parlimen pkr yang enggan dinamakan yang mendakwa azmin',
 'isu bil tnb dibawa ke kabinet - saifuddin',
 'dengan membuat sampel merasa kenyang lebih cepat.',
 'banyak orang yang merasa takut bahwa ai akan menggeser peran manusia.',
 'singapura, menurut kementerian transportasi singapura, tidak akan ragu untuk mengambil tindakan tegas atas aktivitas tersebut.']

In [16]:
encoder = Encoder(bpe)

In [17]:
encoder.encode([train_X[0]])

[[3494,
  1223,
  9869,
  5732,
  8213,
  325,
  25124,
  1826,
  1117,
  1608,
  6951,
  217,
  12107,
  9526,
  217,
  6700,
  13566,
  1]]

In [18]:
string = train_X[0]
encoded = encoder.encode([string])
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: encoded, model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0].tolist()), encoder.decode(b[0].tolist())

2022-08-02 14:58:08.965217: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11


('لاڤورن سبواه ڤورتل بريتا بارو-بارو اين ممتيق سأورڠ اهلي ڤرليمين ڤقر يڠ اڠڬن ديناماكن يڠ مندعوا عزمين<UNK>',
 'لاڤورن سبواه ڤورتل بريتا بارو-بارو اين ممتيق سأورڠ اهلي ڤرليمين ڤقر يڠ اڠڬن ديناماكن يڠ مندعوا عزمين<UNK>')

In [19]:
string = train_X[1]
encoded = encoder.encode([string])
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: encoded, model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0].tolist()), encoder.decode(b[0].tolist())

('ايسو بيل تنب دباوا ك كابينيت - صيفالدين<UNK>',
 'ايسو بيل تنب دباوا ك كابينيت - صيفالدين<UNK>')

In [20]:
string = train_X[2]
encoded = encoder.encode([string])
f, b = sess.run([model.fast_result, model.beam_result], feed_dict = {model.X: encoded, model.temperature: 0.5,
                                                                    model.beam_size: 2})
encoder.decode(f[0].tolist()), encoder.decode(b[0].tolist())

('دڠن ممبوات سمڤل مراسا كينياڠ لبيه چڤت.<UNK>',
 'دڠن ممبوات سمڤل مراسا كينياڠ لبيه چڤت.<UNK>')

In [21]:
f[0][f[0] > 3].tolist()

[358, 2170, 20123, 17179, 10285, 38, 696, 7991, 46]

In [22]:
encoder.decode([0,1,2,3])

'<PAD><UNK><BOS><EOS>'

In [23]:
def calculate_cer(actual, hyp):
    """
    Calculate CER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    actual = actual.replace(' ', '')
    hyp = hyp.replace(' ', '')
    return Lev.distance(actual, hyp) / len(actual)


def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [24]:
string = train_X[1]
encoded = encoder.encode([string])
f = sess.run(model.fast_result, feed_dict = {model.X: encoded})[0]
p = encoder.decode(f[f > 3].tolist())

In [25]:
calculate_cer(train_Y[1], p), calculate_wer(train_Y[1], p)

(0.0, 0.0)

In [26]:
from tqdm import tqdm

cers, wers = [], []
for i in tqdm(range(len(train_X[:10000]))):
    string = train_X[i]
    encoded = encoder.encode([string])
    f = sess.run(model.fast_result, feed_dict = {model.X: encoded})[0]
    p = encoder.decode(f[f > 3].tolist())
    cer, wer = calculate_cer(train_Y[i], p), calculate_wer(train_Y[i], p)
    cers.append(cer)
    wers.append(wer)

100%|██████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [16:02<00:00, 10.39it/s]


In [28]:
import numpy as np

np.mean(cers), np.mean(wers)

(0.0006167541656054869, 0.0019283112815117458)

In [29]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'rumi-jawi-small/model.ckpt')

'rumi-jawi-small/model.ckpt'

In [30]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name
        or 'temperature' in n.name
        or 'beam_size' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'beam_size',
 'temperature',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/bod

In [31]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [32]:
freeze_graph('rumi-jawi-small', strings)

2022-08-02 15:28:50.978444: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 15:28:50.979144: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 Ti major: 8 minor: 6 memoryClockRate(GHz): 1.86
pciBusID: 0000:01:00.0
2022-08-02 15:28:50.979165: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-08-02 15:28:50.979185: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-08-02 15:28:50.979193: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-08-02 15:28:50.979200: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library

INFO:tensorflow:Restoring parameters from rumi-jawi-small/model.ckpt


INFO:tensorflow:Restoring parameters from rumi-jawi-small/model.ckpt


INFO:tensorflow:Froze 81 variables.


INFO:tensorflow:Froze 81 variables.


INFO:tensorflow:Converted 81 variables to const ops.


INFO:tensorflow:Converted 81 variables to const ops.


4441 ops in the final graph.


In [33]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [34]:
g = load_graph('rumi-jawi-small/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
test_sess = tf.InteractiveSession(graph = g)


2022-08-02 15:29:01.286624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-02 15:29:01.287317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1666] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3090 Ti major: 8 minor: 6 memoryClockRate(GHz): 1.86
pciBusID: 0000:01:00.0
2022-08-02 15:29:01.287352: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-08-02 15:29:01.287372: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-08-02 15:29:01.287380: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2022-08-02 15:29:01.287387: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic librar

In [35]:
from tensorflow.tools.graph_transforms import TransformGraph

In [36]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'rumi-jawi-small/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

print(pb)

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder', 'beam_size', 'temperature'],
                                       ['greedy', 'beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

rumi-jawi-small/frozen_model.pb


2022-08-02 15:29:11.126755: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-08-02 15:29:11.146600: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-08-02 15:29:11.165985: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-08-02 15:29:11.166023: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-08-02 15:29:11.190569: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-08-02 15:29:11.190607: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-08-02 15:29:11.212373: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for greedy
2022-08-02 15:29:11.212409: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for beam
2022-08-02 15:29:11.233585: I tensorflow/tools/graph_transforms/remove_nodes.cc:

In [37]:
from malaya_boilerplate.huggingface import upload_dict

In [38]:
!cp t2t-rumi-jawi/small/*.tfevents.* rumi-jawi-small
!tar -cvf rumi-jawi-small.tar rumi-jawi-small

rumi-jawi-small/
rumi-jawi-small/checkpoint
rumi-jawi-small/frozen_model.pb.quantized
rumi-jawi-small/events.out.tfevents.1659413245.huseincomel-desktop
rumi-jawi-small/events.out.tfevents.1659413066.huseincomel-desktop
rumi-jawi-small/model.ckpt.index
rumi-jawi-small/model.ckpt.data-00000-of-00001
rumi-jawi-small/model.ckpt.meta
rumi-jawi-small/frozen_model.pb


In [39]:
files_mapping = {'rumi-jawi-small.tar': 'rumi-jawi-small.tar'}
upload_dict(model = 'pretrained-rumi-jawi', files_mapping = files_mapping)

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.8. Pass `repo_id` instead.
  warnings.warn(


In [40]:
files_mapping = {'rumi-jawi-small/frozen_model.pb': 'model.pb'}
upload_dict(model = 'rumi-jawi-small', files_mapping = files_mapping)

In [41]:
files_mapping = {'rumi-jawi-small/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'rumi-jawi-small-quantized', files_mapping = files_mapping)